<a href="https://colab.research.google.com/github/leodzung/lyft/blob/main/lyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colab setup
https://www.kaggle.com/kool777/ultimate-google-colab-training-batch-size-64

In [2]:
import os

os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/')
os.makedirs('results/multi_train', exist_ok=True)

%cd '/content/lyft-motion-prediction-autonomous-vehicles/'
!pwd

/content
/content/lyft-motion-prediction-autonomous-vehicles


In [3]:
os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/scenes/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/scenes/')

!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/compressed/scenes.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1602291212&Signature=Tu4l72qBB9B5Ip7GghxAV5zkstxEBbZJM8ToEr8Vae9w1M5HLKl0G67wFtFIyglk6yZDJYeODbWtWeLPwS3Vr2oS8GHzZuGTlaiUqDiPrEx7kNDOaik6xx2msNaItnLiKFJ6LZUN%2F8eCpXJEfStfSUF%2BP0tv53KtV29aefXTM2BBhncYWpnCVxPmUpamk9nxSnV7kMqIezXY5GbPmSuwagZg3e0fATeko9tj4pT8eq9yLrMexTiWnAJk9BDkSg6UbaqKueylXIVxlVKgwxBtWSqCbxjhgSh27XXL%2FxC1J4KWXI7RLFgNQeJ55GsetJvRkw801CLgo8LVTebFL3Sw3g%3D%3D&response-content-disposition=attachment%3B+filename%3Dscenes.zip" -c -O 'scenes.zip'
!unzip ./scenes.zip

Streaming output truncated to the last 5000 lines.
  inflating: validate.zarr/agents_mask/0.5/8502.0  
  inflating: validate.zarr/agents_mask/0.5/8503.0  
  inflating: validate.zarr/agents_mask/0.5/8504.0  
  inflating: validate.zarr/agents_mask/0.5/8505.0  
  inflating: validate.zarr/agents_mask/0.5/8506.0  
  inflating: validate.zarr/agents_mask/0.5/8507.0  
  inflating: validate.zarr/agents_mask/0.5/8508.0  
  inflating: validate.zarr/agents_mask/0.5/8509.0  
  inflating: validate.zarr/agents_mask/0.5/851.0  
  inflating: validate.zarr/agents_mask/0.5/8510.0  
  inflating: validate.zarr/agents_mask/0.5/8511.0  
  inflating: validate.zarr/agents_mask/0.5/8512.0  
  inflating: validate.zarr/agents_mask/0.5/8513.0  
  inflating: validate.zarr/agents_mask/0.5/8514.0  
  inflating: validate.zarr/agents_mask/0.5/8515.0  
  inflating: validate.zarr/agents_mask/0.5/8516.0  
  inflating: validate.zarr/agents_mask/0.5/8517.0  
  inflating: validate.zarr/agents_mask/0.5/8518.0  
  inflating: v

In [4]:
%cd '/content/lyft-motion-prediction-autonomous-vehicles/scenes/'
!pwd
!ls

/content
/content/lyft-motion-prediction-autonomous-vehicles/scenes
mask.npz  sample.zarr  scenes.zip  test.zarr  train.zarr  validate.zarr


In [5]:
%rm -r sample.zarr/
%rm scenes.zip

In [6]:
%cd ..
!pwd
!ls

/content/lyft-motion-prediction-autonomous-vehicles
/content/lyft-motion-prediction-autonomous-vehicles
results  scenes


In [7]:
os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/aerial_map/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/aerial_map/')

In [8]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/compressed/aerial_map.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1600609941&Signature=E7FIEyReGbz0syet6Df%2BKAcgUBE3uXyjEG%2FHnTxJnw7oKGyWXSbHPr2P0u8sxxtXnz6efYDkxU3y4UavkimcQIN23QNnelF4u%2FSYd0z3uC6930q9gyhfD7a4CLDAFbkrZMSL4I%2B5vvrKMxcDRUth0%2BWKite7MmURyfOnOUMThunNWEfxYSDSObrRCtYtC%2BXwOmaJz37tD8f78adqLHH3QJyZxS0RhIzDCCk%2BRHE43JnnJ42jeZMSGvBQmcrnovGTCIgiJSdMs9%2Fk1LcBUAor%2BIE%2FeoQYTClw6WEu%2B0f%2BLYzbPt2kr%2BZFswnHr%2BRDCZIIWHWV15%2Fr47c28mt7fzsKlg%3D%3D&response-content-disposition=attachment%3B+filename%3Daerial_map.zip" -c -O 'aerial_map.zip'
!unzip ./aerial_map.zip

--2020-10-07 01:06:04--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/compressed/aerial_map.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1600609941&Signature=E7FIEyReGbz0syet6Df%2BKAcgUBE3uXyjEG%2FHnTxJnw7oKGyWXSbHPr2P0u8sxxtXnz6efYDkxU3y4UavkimcQIN23QNnelF4u%2FSYd0z3uC6930q9gyhfD7a4CLDAFbkrZMSL4I%2B5vvrKMxcDRUth0%2BWKite7MmURyfOnOUMThunNWEfxYSDSObrRCtYtC%2BXwOmaJz37tD8f78adqLHH3QJyZxS0RhIzDCCk%2BRHE43JnnJ42jeZMSGvBQmcrnovGTCIgiJSdMs9%2Fk1LcBUAor%2BIE%2FeoQYTClw6WEu%2B0f%2BLYzbPt2kr%2BZFswnHr%2BRDCZIIWHWV15%2Fr47c28mt7fzsKlg%3D%3D&response-content-disposition=attachment%3B+filename%3Daerial_map.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-10-07 01:06:05 ERROR 400: Bad Request.

Archive:  ./aeri

In [9]:
%cd aerial_map
%rm -r aerial_map.zip

[Errno 2] No such file or directory: 'aerial_map'
/content/lyft-motion-prediction-autonomous-vehicles


In [10]:
os.makedirs('/content/lyft-motion-prediction-autonomous-vehicles/semantic_map/', exist_ok=True)
os.chdir('/content/lyft-motion-prediction-autonomous-vehicles/semantic_map/')

!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/upload/semantic_map.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1600609978&Signature=ZHiPGpBCn49OQ2xKoumVNltmG4DGJvSjWmFT8vogitqo64%2F4abms6Iq%2BGWtDsGIElcd3mD7T%2BbZgoGyWXUUh5bEvHJepsJuAU0Lg44xF30c8%2FkLq3Yg6YoSbHJX%2F8gnNzZ7HM2BIhaqXWlXo7%2BdWJyW%2FFj6tpwCYZxWeu8WuHtf51NW6zJ4n9pxmpbxGzwAH11AwXedO3RPNTN7LpeZ6In5%2Fb54mh%2F%2FuY2uawgPc%2FTetM982qrtPJsHO6rq%2BdZ4Qdj0IROHzn%2F3xvohE5l3iASNQNwEzXCy9fVsIFfNccVMZEDZ486XzoKi%2BvzkF08JDRvL8bORvRsexh2Dlz5aeww%3D%3D&response-content-disposition=attachment%3B+filename%3Dsemantic_map.zip" -c -O 'semantic_map.zip'
!unzip ./semantic_map.zip

--2020-10-07 01:06:05--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19990/1472735/upload/semantic_map.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1600609978&Signature=ZHiPGpBCn49OQ2xKoumVNltmG4DGJvSjWmFT8vogitqo64%2F4abms6Iq%2BGWtDsGIElcd3mD7T%2BbZgoGyWXUUh5bEvHJepsJuAU0Lg44xF30c8%2FkLq3Yg6YoSbHJX%2F8gnNzZ7HM2BIhaqXWlXo7%2BdWJyW%2FFj6tpwCYZxWeu8WuHtf51NW6zJ4n9pxmpbxGzwAH11AwXedO3RPNTN7LpeZ6In5%2Fb54mh%2F%2FuY2uawgPc%2FTetM982qrtPJsHO6rq%2BdZ4Qdj0IROHzn%2F3xvohE5l3iASNQNwEzXCy9fVsIFfNccVMZEDZ486XzoKi%2BvzkF08JDRvL8bORvRsexh2Dlz5aeww%3D%3D&response-content-disposition=attachment%3B+filename%3Dsemantic_map.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-10-07 01:06:05 ERROR 400: Bad Request.

Archive:  ./semant

In [11]:
%rm -r semantic_map.zip
%cd ..
!pwd
!ls

/content/lyft-motion-prediction-autonomous-vehicles
/content/lyft-motion-prediction-autonomous-vehicles
aerial_map  results  scenes  semantic_map


In [12]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  26926      0 --:--:-- --:--:-- --:--:-- 26926
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
     |████████████████████████████████| 61kB 3.2MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     


In [13]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=aafa89dac74dc768f5985d594e495a502e0c84bbe2f3cdc54b972003cfa36b26
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [14]:
!pip install transformers

     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 890kB 16.9MB/s 
     |████████████████████████████████| 1.1MB 26.6MB/s 
     |████████████████████████████████| 3.0MB 34.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0ae3a97b3dc82514431f29177a70afee89283ff5c108a706aaf4981b3409e75b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [15]:
# this script transports l5kit and dependencies
!pip -q install pymap3d==2.1.0 
!pip -q install protobuf==3.12.2 
!pip -q install transforms3d 
!pip -q install zarr 
!pip -q install ptable

!pip -q install --no-dependencies l5kit
!pip install pytorch-pfn-extras==0.2.1
!pip install pytorch-ignite

# VERSION = "1.5"
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

     |████████████████████████████████| 1.3MB 3.5MB/s 
     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 133kB 3.4MB/s 
     |████████████████████████████████| 5.8MB 5.0MB/s 
     |████████████████████████████████| 92kB 2.9MB/s 
     |████████████████████████████████| 51kB 2.0MB/s 
  Created wheel for pytorch-pfn-extras: filename=pytorch_pfn_extras-0.2.1-cp36-none-any.whl size=61397 sha256=c065045637fa36dc53b766a609c7055e14afabcfeaa3572d10c70d9a494bb253
  Stored in directory: /root/.cache/pip/wheels/f2/f4/bd/8dd8508508c6f13860519733af7082a77af283aa0dc464b619
Successfully built pytorch-pfn-extras
     |████████████████████████████████| 184kB 3.4MB/s 


<body>
    <p style="color:magenta; font-size:20px; font-family:sanserif;">Imports</p>
</body>

In [16]:
# # Declaring the path to load efficientNet models.
# import sys
# sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

In [17]:
#IMPORTS
import os
import gc
import time
import numpy as np
import pandas as pd
from scipy import stats
from tqdm.auto import tqdm
from typing import Dict
import matplotlib.pyplot as plt
from tempfile import gettempdir
from prettytable import PrettyTable
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule

# TORCH
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader

# TORCH XLA
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils

# L5Kit
from l5kit.configs import load_config_data
from l5kit.geometry import transform_points
from l5kit.rasterization import build_rasterizer
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset

# EfficientNet
from efficientnet_pytorch import model as enet

# Filter warnings
import warnings
warnings.filterwarnings("ignore")

# TPU ResNet50
https://www.kaggle.com/doanquanvietnamca/tpu-resnet50-faster-better

# Configurations

In [18]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
dm = LocalDataManager(None)
# get config
cfg = {
    'format_version': 4,
    'model_params': {
        'history_num_frames': 15,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [331, 331],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 8,
        'shuffle': True,
        'num_workers': 1
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 12,
        'shuffle': False,
        'num_workers': 1
    }

}

TRAIN_MODE = True

In [19]:
# ===== INIT DATASET
if TRAIN_MODE:
    train_cfg = cfg["train_data_loader"]
    rasterizer = build_rasterizer(cfg, dm)
    train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
    train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], 
                                  batch_size=train_cfg["batch_size"], 
                                 num_workers=train_cfg["num_workers"])
    print(train_dataset)

In [20]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

In [21]:
import torch
import torch.nn as nn

class LyftModel(nn.Module):
    def __init__(self, cfg):
        super(LyftModel, self).__init__()
        # load pre-trained Conv2D model
        self.backbone = resnet18(pretrained=False, progress=False)
        self.backbone.load_state_dict(
            torch.load(
                '../input/resnet18/resnet18.pth'
            )
        )
        # change input channels number to match the rasterizer's output
        num_history_channels = (cfg['model_params']['history_num_frames']+1) * 2
        num_in_channels = 3 + num_history_channels
        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False
        )
        
        # change output size to (X, Y) * number of future states
        num_targets = 2 * cfg["model_params"]["future_num_frames"]
        self.backbone.fc = nn.Linear(in_features=512, out_features=num_targets)        
    
    def forward(self, x):
        # Forward pass
        return self.backbone(x)

In [22]:
def build_model(cfg: Dict) -> torch.nn.Module:
    # load pre-trained Conv2D model
    model = LyftModel(cfg)
    ckpt = torch.load('../input/lyftmodelall/resnet18_15hist_331px.pth', map_location=torch.device('cpu'))
    model.load_state_dict(ckpt)
    return model

build_model(cfg)

In [23]:
class AverageMeter(object):
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [24]:
def train_loop_fn(train_loader, model, optimizer, device, scheduler, epoch=None):
    # Train
    batch_time = AverageMeter('Time', ':6.1f')
    data_time = AverageMeter('Data', ':6.1f')
    losses = AverageMeter('Loss', ':6.1e')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses],
        prefix="[xla:{}]Train:  Epoch: [{}]".format(xm.get_ordinal(), epoch)
    )
    criterion = nn.MSELoss(reduction="none")
    model.train()
    end = time.time()
    best = 1000
    for i, data in enumerate(train_loader):
        data_time.update(time.time()-end)
        inputs = data["image"].to(device)
        target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
        targets = data["target_positions"].to(device)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs).reshape(targets.shape)
        loss = criterion(outputs, targets)
        # not all the output steps are valid, but we can filter them out from the loss using availabilities
        loss = loss * target_availabilities
        loss = loss.mean()
        loss.backward()
        xm.optimizer_step(optimizer)
        losses.update(loss.item(), inputs.size(0))
        scheduler.step(metrics=loss)
        batch_time.update(time.time() - end)
        end = time.time()
        
        if i % 50 == 0:
            progress.display(i)
            if losses.avg< best:
                best = losses.avg 
                xm.save(model.state_dict(), "model.bin")
                xm.master_print(f'Model Saved.{best}')
        
    del loss
    del outputs
    gc.collect()

Model Loaded on xla:1
Preparing the dataloader..


Exception in device=TPU:0: semantic_map/semantic_map.pb is not present in local data folder /content/lyft-motion-prediction-autonomous-vehicles
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 228, in _start_fn
    fn(gindex, *args)
  File "<ipython-input-24-88d958e87356>", line 22, in train
    train_dataloader = get_dataloader(cfg["train_data_loader"], dm.require("scenes/train.zarr"))
  File "<ipython-input-22-178f8af7c980>", line 5, in get_dataloader
    rasterizer = build_rasterizer(cfg, dm)
  File "/usr/local/lib/python3.6/dist-packages/l5kit/rasterization/rasterizer_builder.py", line 155, in build_rasterizer
    semantic_map_filepath = data_manager.require(raster_cfg["semantic_map_key"])
  File "/usr/local/lib/python3.6/dist-packages/l5kit/data/local_data_manager.py", line 57, in require
    raise FileNotFoundError(f"{key} is not present in local data folder {self.root_folder}")
FileNotFoundError

Exception: ignored

In [ ]:
from torch.utils.data.distributed import DistributedSampler
import torch_xla.debug.metrics as met
WRAPPED_MODEL = xmp.MpModelWrapper(build_model(cfg))

def _run():
    TRAIN_BATCH_SIZE = 8
    EPOCHS = 1
    xm.master_print('Starting Run ...')
    train_sampler = DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False
    )
    
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=0
    )
    xm.master_print('Train Loader Created.')
    
    num_train_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE / xm.xrt_world_size())
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)
    xm.master_print('Done Model Loading.')
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                           factor=0.8, patience=3, 
                                                           verbose=False, 
                                                           threshold=0.0001, min_lr=0.00001)
    xm.master_print(f'Num Train Steps= {num_train_steps}, XRT World Size= {xm.xrt_world_size()}.')
    
    for epoch in range(EPOCHS):
        para_loader = tlp.ParallelLoader(train_data_loader, [device])
        xm.master_print('Parallel Loader Created. Training ...')
        train_loop_fn(para_loader.per_device_loader(device),
                      model,  
                      optimizer, 
                      device, 
                      scheduler, 
                      epoch
                     )
        
        xm.master_print("Finished training epoch {}".format(epoch))
        if epoch == EPOCHS-1:
            xm.master_print('Saving Model ..')
            xm.save(model.state_dict(), "model.bin")
            xm.master_print('Model Saved.')
            
    if METRICS_DEBUG:
      xm.master_print(met.metrics_report(), flush=True)

In [ ]:
import time
from torch.nn import functional as F
def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    _run()

FLAGS={}
if TRAIN_MODE:
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=1, start_method='fork')

# Test Mode

In [ ]:
import numpy as np
import os
import torch

from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from typing import Dict

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.evaluation import write_pred_csv
from l5kit.rasterization import build_rasterizer

In [ ]:
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

SINGLE_MODE_SUBMISSION = f"{DIR_INPUT}/single_mode_sample_submission.csv"
MULTI_MODE_SUBMISSION = f"{DIR_INPUT}/multi_mode_sample_submission.csv"

# Training notebook's output.

In [ ]:
model = LyftModel(cfg)
ckpt = torch.load('../input/lyftmodelall/resnet18_15_331px.bin')
model.load_state_dict(ckpt)

In [ ]:
# ===== INIT DATASET
test_cfg = cfg["test_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataloader)

In [ ]:
def _test():
    

    future_coords_offsets_pd = []
    timestamps = []
    agent_ids = []
    device = 'xla:0'
    print(f"device: {device} ready!")
    model = LyftModel(cfg)
    ckpt = torch.load('../input/lyftmodelall/resnet18_15_331px.bin')
    model.load_state_dict(ckpt)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        dataiter = tqdm(test_dataloader)

        for data in dataiter:
            inputs = data["image"].to(device)
            target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
            targets = data["target_positions"].to(device)

            outputs = model(inputs).reshape(targets.shape)

            future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
            timestamps.append(data["timestamp"].numpy().copy())
            agent_ids.append(data["track_id"].numpy().copy())
    write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd))

In [ ]:
def _mp_fn(rank, flags):
#     torch.set_default_tensor_type('torch.FloatTensor')
    _test()

FLAGS={}
if not TRAIN_MODE:
    xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=1, start_method='fork')

In [ ]:
import pandas as pd

# Change these to your dataset/submission.csv
SUBMISSION_FOLDER = '../input/resnet4days'
SUBMISSION_FILE = 'resnet50-10000000.csv' #keras resnet50


submissions = pd.read_csv(f"{SUBMISSION_FOLDER}/{SUBMISSION_FILE}")
submissions.to_csv("submission.csv", index=False)